## Исследование clickhouse

## Подготовка базы данных

In [43]:
from clickhouse_driver import Client

client = Client(host='localhost')

In [44]:
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',),
 ('analysis',),
 ('default',),
 ('example',),
 ('information_schema',),
 ('system',)]

In [45]:
# Создаем базу для тестирования
create_db_query = "CREATE DATABASE IF NOT EXISTS analysis ON CLUSTER company_cluster" 

client.execute(create_db_query)

[('clickhouse-node1', 9000, 0, '', 3, 0),
 ('clickhouse-node3', 9000, 0, '', 2, 0),
 ('clickhouse-node2', 9000, 0, '', 1, 0),
 ('clickhouse-node4', 9000, 0, '', 0, 0)]

In [58]:
# Создаем таблицу для тестирования
create_table_query = "CREATE TABLE analysis.test_progress ON CLUSTER company_cluster (id String, type String, move_id String, movie_timestamp Int64) Engine=MergeTree() ORDER BY id"

client.execute(create_table_query)

## Подготовка скриптов для загрузки данных

In [59]:
import uuid
from random import randint


def generate_progress_data(count: int):
    chunk = []
    
    for i in range(count):
        movie_timestamp = randint(1, 1_000_000_000)
        chunk.append((str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), movie_timestamp))

    return chunk

In [187]:
import time
import statistics

class TimerCode():
    def __init__(self):
        self.start = time.time()
        self.durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.durations.append(end_time - self.start)
        self.start = end_time
    
    def get_current_stat(self):
        print(f"Median - {statistics.median(self.durations)}")
        avg = sum(self.durations) / len(self.durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.durations)}")
        
    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.get_current_stat()

## Тестирование вставки данных

In [61]:
# тестируем вставку данных списком из 100 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(100)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.05293893814086914
Average - 0.05293893814086914
Summary - 0.05293893814086914


In [108]:
# тестируем вставку данных списком из 250 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(250)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.016029834747314453
Average - 0.016029834747314453
Summary - 0.016029834747314453


In [107]:
# тестируем вставку данных списком из 500 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(500)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.0930318832397461
Average - 0.0930318832397461
Summary - 0.0930318832397461


In [109]:
# тестируем вставку данных списком из 850 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(850)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.10680294036865234
Average - 0.10680294036865234
Summary - 0.10680294036865234


In [110]:
# тестируем вставку данных списком из 1_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(1_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.0521237850189209
Average - 0.0521237850189209
Summary - 0.0521237850189209


In [78]:
# тестируем вставку данных списком из 10_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(10_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.27316999435424805
Average - 0.27316999435424805
Summary - 0.27316999435424805


In [68]:
# тестируем вставку данных списком из 50_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(50_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 0.34465885162353516
Average - 0.34465885162353516
Summary - 0.34465885162353516


In [77]:
# тестируем вставку данных списком из 100_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(100_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 1.100376844406128
Average - 1.100376844406128
Summary - 1.100376844406128


In [75]:
# тестируем вставку данных списком из 1_000_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(1_000_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 9.904001712799072
Average - 9.904001712799072
Summary - 9.904001712799072


In [79]:
# тестируем вставку данных списком из 5_000_000 штук

with TimerCode() as timer:
    for _ in range(1):
        chunk = generate_progress_data(5_000_000)
        timer.setup_start_time()
        client.execute(
                    f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
                    chunk,
                )
        timer.checkpoint()

Median - 42.03411412239075
Average - 42.03411412239075
Summary - 42.03411412239075


## Тестирование чтения данных

На момент исследования количество записей в базе

```
SELECT count(*)
FROM analysis.test_progress

Query id: e31af605-edd4-4343-8054-087549e0775c

┌──count()─┐
│ 11360000 │
└──────────┘
```

In [162]:
# тестируем чтение данных 1_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT count(*) FROM analysis.test_progress LIMIT 1000')
        timer.checkpoint()

Median - 0.002722620964050293
Average - 0.004478335380554199
Summary - 0.04478335380554199


In [159]:
# тестируем чтение данных 10_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT count(*) FROM analysis.test_progress LIMIT 10000')
        timer.checkpoint()

Median - 0.0032401084899902344
Average - 0.00474696159362793
Summary - 0.0474696159362793


In [156]:
# тестируем чтение данных 100_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT count(*) FROM analysis.test_progress LIMIT 100000')
        timer.checkpoint()

Median - 0.0041408538818359375
Average - 0.011403751373291016
Summary - 0.11403751373291016


In [155]:
# тестируем чтение данных 1_000_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT count(*) FROM analysis.test_progress LIMIT 1000000')
        timer.checkpoint()

Median - 0.003074049949645996
Average - 0.003217625617980957
Summary - 0.03217625617980957


In [154]:
# тестируем чтение данных 10_000_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT count(*) FROM analysis.test_progress LIMIT 10000000')
        timer.checkpoint()

Median - 0.004253864288330078
Average - 0.006939530372619629
Summary - 0.06939530372619629


## Тестируем чтение + вычисление длины строки

In [153]:
# тестируем чтение данных + вычисление длины строки 1_000

with TimerCode() as timer:
    for _ in range(10):
        timer.setup_start_time()
        client.execute('SELECT length(type) FROM analysis.test_progress limit 1000')
        timer.checkpoint()

Median - 0.010387539863586426
Average - 0.010994052886962891
Summary - 0.1099405288696289


In [96]:
# тестируем чтение данных + вычисление длины строки 100_000

with TimerCode() as timer:
    for _ in range(1):
        timer.setup_start_time()
        client.execute('SELECT length(type) FROM analysis.test_progress limit 100000')
        timer.checkpoint()

Median - 0.09389615058898926
Average - 0.09389615058898926
Summary - 0.09389615058898926


In [104]:
# тестируем чтение данных + вычисление длины строки без лимита

with TimerCode() as timer:
    for _ in range(1):
        timer.setup_start_time()
        client.execute('SELECT length(type) FROM analysis.test_progress')
        timer.checkpoint()

Median - 2.7380447387695312
Average - 2.7380447387695312
Summary - 2.7380447387695312


## Аналитические задачи

In [114]:
# Найдем максимальное значение среди movie_timestamp

with TimerCode() as timer:
    for _ in range(1):
        timer.setup_start_time()
        result = client.execute('SELECT max(movie_timestamp) FROM analysis.test_progress')
        timer.checkpoint()
        
        for item in result:
            print("Максимальная отметка о просмотренном фильме", item)

Максимальная отметка о просмотренном фильме (999999853,)
Median - 0.1840221881866455
Average - 0.1840221881866455
Summary - 0.1840221881866455


In [122]:
# Вычисление энтропии столбца
# https://clickhouse.com/docs/ru/sql-reference/aggregate-functions/reference/entropy

with TimerCode() as timer:
    for _ in range(1):
        timer.setup_start_time()
        result = client.execute('SELECT entropy(movie_timestamp) FROM analysis.test_progress')
        timer.checkpoint()
        
        for item in result:
            print("Энтропия столбца movie_timestamp", item)

Энтропия столбца movie_timestamp (23.426580532703348,)
Median - 5.341540098190308
Average - 5.341540098190308
Summary - 5.341540098190308


In [165]:
# Построение гистрограммы

with TimerCode() as timer:
    timer.setup_start_time()
    result = client.execute("""SELECT histogram(50)(50 + 1) FROM (SELECT * FROM analysis.test_progress)""")
    timer.checkpoint()

    for item in result:
        print(item)

([(51.0, 51.0, 13441100.0)],)
Median - 1.3001930713653564
Average - 1.3001930713653564
Summary - 1.3001930713653564


In [166]:
# Вычисление квантиля 
# https://clickhouse.com/docs/ru/sql-reference/aggregate-functions/reference/quantiletiming

with TimerCode() as timer:
    timer.setup_start_time()
    result = client.execute("SELECT quantile(movie_timestamp) FROM analysis.test_progress")
    timer.checkpoint()

    for item in result:
        print(item)

(498155918.5,)
Median - 0.16817092895507812
Average - 0.16817092895507812
Summary - 0.16817092895507812


## Тестирование данных поступающих в реальном времени

In [ ]:
def insert_data():
    """Функция занимается постоянной вставкой данных в хранилище."""
    for _ in range(10):
        chunk = generate_progress_data(50_000)
        client.execute(
            f"INSERT INTO analysis.test_progress (id, type, move_id, movie_timestamp) VALUES",
            chunk,
        )
        print('Вставка данных прошла успешна')
        
def select_data():
    """Функция занимается постоянным вычитыванием данных из хранилища."""
    with TimerCode() as timer:
        for _ in range(10):
            timer.setup_start_time()
            client.execute('SELECT length(type) FROM analysis.test_progress limit 1000')
            timer.checkpoint()
            timer.get_current_stat()
            print('Чтение данных прошло успешно!')
            
def run():
    while True:
        insert_data()
        select_data()

run()

## Результат работы

# Чтение данных прошло успешно!
# Median - 0.01027989387512207
# Average - 0.014844179153442383
# Summary - 0.11875343322753906
# Чтение данных прошло успешно!
# Median - 0.010419845581054688
# Average - 0.015707148445977107
# Summary - 0.14136433601379395
# Чтение данных прошло успешно!
# Median - 0.01027989387512207
# Average - 0.01470353603363037
# Summary - 0.1470353603363037
# Чтение данных прошло успешно!
# Median - 0.01027989387512207
# Average - 0.01470353603363037
# Summary - 0.1470353603363037
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Median - 0.03676176071166992
# Average - 0.03676176071166992
# Summary - 0.03676176071166992
# Чтение данных прошло успешно!
# Median - 0.023945331573486328
# Average - 0.023945331573486328
# Summary - 0.047890663146972656
# Чтение данных прошло успешно!
# Median - 0.02214670181274414
# Average - 0.023345788319905598
# Summary - 0.0700373649597168
# Чтение данных прошло успешно!
# Median - 0.016637802124023438
# Average - 0.019518792629241943
# Summary - 0.07807517051696777
# Чтение данных прошло успешно!
# Median - 0.012897014617919922
# Average - 0.01819443702697754
# Summary - 0.0909721851348877
# Чтение данных прошло успешно!
# Median - 0.01204061508178711
# Average - 0.017026066780090332
# Summary - 0.10215640068054199
# Чтение данных прошло успешно!
# Median - 0.011184215545654297
# Average - 0.016123192650931224
# Summary - 0.11286234855651855
# Чтение данных прошло успешно!
# Median - 0.01204061508178711
# Average - 0.015736043453216553
# Summary - 0.12588834762573242
# Чтение данных прошло успешно!
# Median - 0.011184215545654297
# Average - 0.01483260260687934
# Summary - 0.13349342346191406
# Чтение данных прошло успешно!
# Median - 0.01204061508178711
# Average - 0.014876842498779297
# Summary - 0.14876842498779297
# Чтение данных прошло успешно!
# Median - 0.01204061508178711
# Average - 0.014876842498779297
# Summary - 0.14876842498779297
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Вставка данных прошла успешна
# Median - 0.005998134613037109
# Average - 0.005998134613037109
# Summary - 0.005998134613037109